In [1]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split as tts
from sklearn.metrics import precision_recall_curve, average_precision_score, roc_curve, roc_auc_score
from sklearn.preprocessing import StandardScaler as scaler
from sklearn.linear_model import LogisticRegression

In [2]:
df = pd.read_excel('data.xlsx')

In [3]:
df

,target,ps_ind_01,ps_ind_02_cat,ps_ind_03,ps_ind_04_cat,ps_ind_05_cat,ps_ind_06_bin,ps_ind_07_bin,ps_ind_08_bin,ps_ind_09_bin,...,ps_calc_11,ps_calc_12,ps_calc_13,ps_calc_14,ps_calc_15_bin,ps_calc_16_bin,ps_calc_17_bin,ps_calc_18_bin,ps_calc_19_bin,ps_calc_20_bin
0,1,2,1,5,1,0,0,0,1,0,...,10,0,3,5,0,1,0,0,0,0
1,1,1,1,4,0,0,1,0,0,0,...,6,2,3,9,0,1,0,1,0,1
2,1,2,1,1,1,0,1,0,0,0,...,7,0,2,10,1,1,0,0,0,1
3,1,3,2,1,0,0,0,0,1,0,...,7,1,7,6,0,0,0,0,0,0
4,1,5,1,3,1,0,0,1,0,0,...,5,0,3,11,1,1,1,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19995,0,0,1,3,0,0,1,0,0,0,...,7,1,4,10,0,0,1,0,0,0
19996,0,0,1,6,1,0,0,0,1,0,...,8,2,4,5,1,1,1,0,0,0
19997,0,5,1,0,1,-1,0,1,0,0,...,6,0,2,7,1,0,1,0,0,0
19998,0,2,1,8,0,0,0,0,0,1,...,2,1,1,14,0,0,1,0,1,0


Let's standardize dataset

In [4]:
y = df.target

df.drop(['target'], axis=1)

df_standardized = scaler().fit_transform(df.values)

df_standardized

array([[ 1.        , -0.00587637, -0.54935044, ..., -0.62740028,
        -0.72719889, -0.42140111],
       [ 1.        , -0.49762239, -0.54935044, ...,  1.5938788 ,
        -0.72719889,  2.37303597],
       [ 1.        , -0.00587637, -0.54935044, ..., -0.62740028,
        -0.72719889,  2.37303597],
       ...,
       [-1.        ,  1.46936172, -0.54935044, ..., -0.62740028,
        -0.72719889, -0.42140111],
       [-1.        , -0.00587637, -0.54935044, ..., -0.62740028,
         1.37513962, -0.42140111],
       [-1.        , -0.49762239, -0.54935044, ..., -0.62740028,
         1.37513962, -0.42140111]])

Split our standardized dataset to train and test dataframes

In [5]:
x_train, x_test, y_train, y_test = tts(df_standardized, y, test_size=0.3)

Fit a model

In [13]:
clf = LogisticRegression()

model = clf.fit(x_train, y_train)
predictions = clf.predict(x_test)

clf.predict_proba(x_test)

array([[5.26943931e-04, 9.99473056e-01],
       [9.99574398e-01, 4.25601848e-04],
       [9.99476555e-01, 5.23444630e-04],
       ...,
       [7.08562921e-04, 9.99291437e-01],
       [5.53462701e-04, 9.99446537e-01],
       [9.99467344e-01, 5.32656418e-04]])

In [16]:
clf.score(x_test, y_test)

1.0